In [22]:
# import os
# api_key = os.getenv("OPENAI_API_KEY")

## Import input data

In [4]:
def read_file(file_path):
    with open(file_path) as f:
        return f.readlines()

# Read job_info.txt,profile_info.txt,letter_raw.txt files
job_info = read_file("job_info.txt")
profile_info = read_file("profile_info.txt")
letter_raw = read_file("letter_raw.txt")

## Open AI API

### Function to call the OpenAI API

In [9]:
def create_openai_completion(model, messages):
    from openai import OpenAI
    client = OpenAI()
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature= 0.4
        )
        return completion
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Function to prepare the prompt for the API

In [6]:

def prepare_messages(profile_info, job_info, letter_format):
    return [
        {"role": "system", "content": "Je suis un assistant qui permet de remplir les lettres de motivation pour vos candidatures. Je fais le lien entre vos expériences, vos compétences et l'offre d'emploi."},
        {"role": "user", "content": f"Voici mes compétences et expériences : {profile_info}"},
        {"role": "system", "content": "Donnez moi l'offre d'emploi."},
        {"role": "user", "content": f"Voici le contenu de l'offre d'emploi : {job_info}"},
        {"role": "system", "content": "Donne moi le format type de lettre de motivation que je dois suivre."},
        {"role": "user", "content": f"Voici le format type que j'aimerais que tu suives : {letter_format}"},
    ]

### Function to get the response from the API

In [10]:
messages = prepare_messages(profile_info, job_info, letter_raw)
completion = create_openai_completion("gpt-4-0125-preview", messages)

if completion:
    result_raw = completion.choices[0].message.content
    print(completion.choices[0].message.content)
else:
    print("Failed to get a response from the OpenAI API.")

Poste & Entreprise : Bouygues Telecom - Alternance Développeur IA/Test F/H
Objet : Candidature pour un stage en Alternance Développeur IA/Test F/H chez Bouygues Telecom

Madame, Monsieur,

Actuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cet expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Alternance Développeur IA/Test F/H.

Votre entreprise est reconnue pour son expertise en matière de technologies de pointe et je suis convaincu que Bouygues Telecom est l'endroit idéal pour développer mes compétences. L'engagement de Bouygues Telecom dans l'innovation, notamment dans le domaine de l'intelligence artificielle et des systèmes d'informations, est particulièrement inspirant po

## Data Cleaning

In [11]:
def extract_text_between_markers(text, start_marker, end_marker):
    try:
        start_index = text.find(start_marker)
        if start_index == -1:
            raise ValueError(f"Start marker '{start_marker}' not found in text.")

        start_index += len(start_marker)
        end_index = text.find(end_marker, start_index)
        if end_index == -1:
            # If end marker is not found, it's not always an error.
            return text[start_index:].strip()

        return text[start_index:end_index].strip()
    except Exception as e:
        # Log the error.
        print(f"An error occurred while extracting text: {e}")
        # Decide on the return value in case of error. None, an empty string, or some default value might be appropriate.
        return None

In [14]:
# Extracting segments
poste_entreprise = extract_text_between_markers(result_raw, "Poste & Entreprise : ", "\nObjet")
letter = extract_text_between_markers(result_raw, "Objet : ", "DESCLOUS") + " DESCLOUS"

print(poste_entreprise)
print(letter)

Bouygues Telecom - Alternance Développeur IA/Test F/H
Candidature pour un stage en Alternance Développeur IA/Test F/H chez Bouygues Telecom

Madame, Monsieur,

Actuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cet expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Alternance Développeur IA/Test F/H.

Votre entreprise est reconnue pour son expertise en matière de technologies de pointe et je suis convaincu que Bouygues Telecom est l'endroit idéal pour développer mes compétences. L'engagement de Bouygues Telecom dans l'innovation, notamment dans le domaine de l'intelligence artificielle et des systèmes d'informations, est particulièrement inspirant pour moi qui aspire à contribue

In [ ]:
link_to_the_job = job_info[0]
export_content = f"{link_to_the_job}\n\n{letter}"

## Export result

In [27]:
# Write the content to CL/CL - [Position] - [Company].txt
file_path = f"CL/CL - {poste_entreprise.replace('/', '-')}.txt"
with open(file_path, "w") as file:
    file.write(export_content)